In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

### Load data

Circuit tuning data (`data`)

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'
metadata_path = rd.datadir/'projects'/'miR-iFFL'/'plasmids'
data, quantiles, stats, metadata = base.load_data(base_path, metadata_path, 'tuning', 'designs')

In [ ]:
# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
designs_palette = metadata_dict['color']
designs_markers = metadata_dict['markers']

ts_label = {'na': 'base', 'NT': 'OL', 'T': 'CL', 'none': '–'}
metadata['ts_label'] = metadata['ts_kind'].replace(ts_label)

Two-gene data (`data2`)

In [ ]:
data2, quantiles2, stats2, metadata2 = base.load_data(base_path, metadata_path, 'two_gene')

In [ ]:
# Create palette for two-gene data
metadata2.loc[(metadata2['gene']=='2V') & (metadata2['construct2_promoter']=='U6'), 'color'] = base.colors['blue']
metadata2.loc[(metadata2['ts_kind']=='NT'), 'color'] = base.colors['gray']
metadata2.loc[metadata2['group'].isin(['base','marker']), 'color'] = 'black'

metadata_dict2 = metadata2.set_index('condition').to_dict('dict')
condition_palette = metadata_dict2['color']
condition_markers = metadata_dict2['markers']

architecture_order = ['1T', '2T', '2V']

In [ ]:
# Create a second palette for regular tuning
metadata3 = base.get_metadata(metadata_path/'construct-metadata.xlsx', 'tuning')
metadata_dict3 = metadata3.set_index('construct').to_dict('dict')
main_palette = metadata_dict3['color']
main_markers = metadata_dict3['markers']

### Set up figure

In [ ]:
base_size = base.font_sizes['base_size']
smaller_size = base.font_sizes['smaller_size']

sns.set_style('ticks')
sns.set_context('paper', font_scale=1.0, rc={'font.size': base_size, 'font.family': 'sans-serif', 'font.sans-serif':['Arial']})
plt.rcParams.update({'axes.titlesize': base_size, 'axes.labelsize': base_size, 'xtick.labelsize': smaller_size, 'ytick.labelsize': smaller_size,
                     'pdf.fonttype': 42, 
                     'ytick.major.size': 3, 'xtick.major.size': 3, 'ytick.minor.size': 2, 'ytick.major.pad': 2, 'xtick.major.pad': 2, 
                     'lines.linewidth': 1,
                     'axes.spines.right': False, 'axes.spines.top': False, 'axes.labelpad': 2})

In [ ]:
# Create the overall figure, gridspec, and add subfigure labels
fig = plt.figure(figsize=(6.8504,8))
fig_gridspec = matplotlib.gridspec.GridSpec(5, 6, figure=fig,
    wspace=0.4, hspace=0.4, height_ratios=(1.75,1.75,1.75,1,1.75),)
subfigures = {
    'A': fig.add_subfigure(fig_gridspec[0,:3]),
    'B': fig.add_subfigure(fig_gridspec[0,3:]),
    'C': fig.add_subfigure(fig_gridspec[1,:3]),
    'D': fig.add_subfigure(fig_gridspec[1,3:]),
    'E': fig.add_subfigure(fig_gridspec[2,:3]),
    'F': fig.add_subfigure(fig_gridspec[2,3:]),
    'G': fig.add_subfigure(fig_gridspec[3,:]),
    'H': fig.add_subfigure(fig_gridspec[4,:]),
    'I': fig.add_subfigure(fig_gridspec[4,:]),
}
for label, subfig in subfigures.items():
    subfig.add_artist(matplotlib.text.Text(x=0, y=1, text=f'{label}', fontsize=base.font_sizes['subpanel_label'], 
                                           fontweight='bold', verticalalignment='top',transform=subfig.transSubfigure))
scatter_kwargs = dict(s=4, jitter=0.2, linewidth=0.5, edgecolor='white')
xlim = (2e2,6e4)
ylim = (2e1,1e5)

output_path = rd.rootdir/'output'/'fig_designs-supp'/'fig_designs-supp.pdf'
fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['A']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.5, top=0.35, right=0.5)
axes = subfig.subplots(1,2, gridspec_kw=dict(wspace=0.45))

plot_df = stats[(((stats['miR']=='miRE.FF4') & (stats['group']=='controller') & (stats['ts_num']==1)) | (stats['group']=='base')) &
                       (stats['promoter']=='EF1a')]

# stat std
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_std', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlim=(-0.5, len(plot_df['design'].unique())-0.5), xlabel='design', ylabel='', yscale='log',)

# stat CV
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='design', y='output_variation', hue='construct', palette=designs_palette,
                    legend=False, ax=ax, marker=designs_markers[construct], **scatter_kwargs)
ax.set(title='CV', xlim=(-0.5, len(plot_df['design'].unique())-0.5), xlabel='', ylabel='',)

for ax in axes: 
    sns.despine(ax=ax)
    ax.set(xticklabels=[l.get_text() if l.get_text()!='0' else 'base' for l in ax.get_xticklabels()])

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['B']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.5, top=0.65, right=0.15)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.35))

# EF1a miR controls
miR_order = ['none', 'miR.FF5', 'miR.FF4', 'miRE.FF5', 'miRE.FF4',]
plot_df = stats[((((stats['group']=='miR')) & (stats['miR_loc']=='UTR')) | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
plot_df['miR'] = plot_df['miR'].astype(pd.api.types.CategoricalDtype(categories=miR_order, ordered=True))
xlim = (-0.5, len(plot_df['construct'].unique())-0.5)
ylim = (2e1,8e4)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=ylim)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=ylim)

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=4, jitter=0.1, linewidth=0.5, edgecolor='white')
ax.set(title='Slope', xlim=xlim, xlabel='', ylim=(0,1.3), ylabel='',
       yticks=[0,0.25,0.5,0.75,1], yticklabels=['0.0','','0.5','','1.0'])
ax.axhline(marker_baseline, color='black', ls=':')

for ax in axes:
    ax.set_xticklabels([l.get_text().replace('.','-') for l in ax.get_xticklabels()], rotation=45, ha='right')
    sns.despine(ax=ax)
    ax.minorticks_off()

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['C']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.5, top=0.65, right=0.15)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.35))

# EF1a miR controls
ts_order = ['none','FF3x1','FF4x1','FF5x1','FF6x1']
plot_df = stats[((((stats['group']=='ts5')) & (stats['ts_num']==1)) | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
plot_df['ts'] = plot_df['ts'].astype(pd.api.types.CategoricalDtype(categories=ts_order, ordered=True))
xlim = (-0.5, len(plot_df['ts'].unique())-0.5)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Mean', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=ylim)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Std.', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=ylim)

# slope
ax = axes[2]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Slope', xlim=xlim, xlabel='', ylim=(0,1.3), ylabel='',
       yticks=[0,0.25,0.5,0.75,1], yticklabels=['0.0','','0.5','','1.0'])
ax.axhline(marker_baseline, color='black', ls=':')

for ax in axes:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    sns.despine(ax=ax)
    ax.minorticks_off()

fig.savefig(rd.outfile(output_path))

In [ ]:
def designs_stats_plots(axes, plot_df):

    xlim = (-0.5, len(plot_df['design'].unique())-0.5)

    # stat gmean
    ax = axes[0]
    for construct, group in plot_df.groupby('construct'):
        sns.stripplot(data=group, x='design', y='output_gmean', hue='construct', palette=designs_palette,
                        legend=False, ax=ax, marker=designs_markers[construct], **scatter_kwargs)
    ax.set(title='Mean', xlim=xlim, xlabel='', ylabel='', yscale='log',)
    marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
    ax.axhline(marker_baseline, color='black', ls=':')
    ax.minorticks_off()

    # stat std
    ax = axes[1]
    for construct, group in plot_df.groupby('construct'):
        sns.stripplot(data=group, x='design', y='output_std', hue='construct', palette=designs_palette,
                        legend=False, ax=ax, marker=designs_markers[construct], **scatter_kwargs)
    ax.set(title='Std.', xlim=xlim, xlabel='design', ylabel='', yscale='log',)

    # slope
    ax = axes[2]
    for construct, group in plot_df.groupby('construct'):
        sns.stripplot(data=group, x='design', y='slope', hue='construct', palette=designs_palette,
                        legend=False, ax=ax, marker=designs_markers[construct], **scatter_kwargs)
    ax.set(title='Slope', xlim=xlim, xlabel='', ylabel='', ylim=(0,1.3),
        yticks=[0,0.25,0.5,0.75,1], yticklabels=['0.0','','0.5','','1.0'])
    ax.axhline(marker_baseline, color='black', ls=':')

    for ax in axes: sns.despine(ax=ax)

In [ ]:
subfig = subfigures['D']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.5, top=0.6, right=0.15)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.45))

plot_df = stats[(stats['miR']=='miR.FF4') & (stats['group']=='controller') & (stats['ts_num']==1) &
                (stats['promoter']=='EF1a')]

designs_stats_plots(axes, plot_df)

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['E']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.5, top=0.6, right=0.15)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.45))

plot_df = stats[(stats['miR']=='miR.FF5') & (stats['group']=='controller') & (stats['ts_num']==1) & 
                (stats['ts'].isin(['FF4x1','FF5x1','FF6x1'])) & (stats['promoter']=='EF1a')]

designs_stats_plots(axes, plot_df)

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['F']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.5, top=0.6, right=0.15)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.45))

plot_df = stats[(stats['miR']=='miRE.FF5') & (stats['group']=='controller') & (stats['ts_num']==1) &
                (stats['promoter']=='EF1a')]

designs_stats_plots(axes, plot_df)

fig.savefig(rd.outfile(output_path))

In [ ]:
# Data for two-gene architectures with 3'UTR target sites
stats_subset = stats2[((stats2['gene']=='1T') & (stats2['design']==1) & (stats2['group']=='controller')) |
                ((stats2['gene']=='2T') & (stats2['group']=='dual') & (stats2['ts_loc']=='3\'')) |
                ((stats2['gene']=='2V') & (stats2['group']=='ts3') & (stats2['construct2_promoter']=='EF1a')) |
                ((stats2['group']=='base') & (stats2['construct2_promoter']!='U6'))].copy()
stats_subset.sort_values(['gene','construct2_promoter','group','ts_kind'], inplace=True)

# Remove outliers (1T base biorep1, 2V EF1a base/NT biorep4, design 2 T biorep1 -- see `flow/outliers.ipynb`)
stats_subset = stats_subset[~((stats_subset['gene']=='1T') & (stats_subset['group']=='base') & (stats_subset['biorep']==1))]
stats_subset = stats_subset[~((stats_subset['gene']=='2V') & (stats_subset['construct2_promoter']=='EF1a') & (stats_subset['ts_kind'].isin(['na','NT'])) & (stats_subset['biorep']==4))]
stats_subset = stats_subset[~((stats_subset['design']==2) & (stats_subset['ts_kind']=='T') & (stats_subset['biorep']==1))]

In [ ]:
# two-gene architectures with 5'UTR target sites
stats_subset = stats2[((stats2['gene']=='1T') & (stats2['design']>1) & (stats2['group']=='controller')) |
                ((stats2['gene']=='2T') & (stats2['group']=='dual') & (stats2['ts_loc']=='5\'')) |
                ((stats2['gene']=='2V') & (stats2['group']=='ts5')) |
                (stats2['group']=='base')].copy()
stats_subset.sort_values(['gene','construct2_promoter','group','ts_kind'], inplace=True)

# Remove outliers (1T base biorep1, 2V EF1a base/NT biorep4 -- see `flow/outliers.ipynb`)
stats_subset = stats_subset[~((stats_subset['gene']=='1T') & (stats_subset['group']=='base') & (stats_subset['biorep']==1))]
stats_subset = stats_subset[~((stats_subset['gene']=='2V') & (stats_subset['construct2_promoter']=='EF1a') & (stats_subset['ts_kind'].isin(['na','NT'])) & (stats_subset['biorep']==4))]
stats_subset = stats_subset[~((stats_subset['design']==2) & (stats_subset['ts_kind']=='T') & (stats_subset['biorep']==1))]

In [ ]:
# Define values to shift xticks in plotting 
# to add more space between architecture groups
buffer = 0.6
gene_order = ['1T', '2T', '2V']
xtick_locs = [0,1,2,3,4, 5+buffer,6+buffer, 7+buffer*2,8+buffer*2,9+buffer*2, 10+buffer*3,11+buffer*3,12+buffer*3,]

condition_loc = {k:v for k,v in zip(stats_subset['condition'].unique(), xtick_locs)}
stats_subset['condition_loc'] = stats_subset['condition'].replace(condition_loc)

metadata2['condition_loc'] = metadata2['condition'].map(condition_loc)
m = metadata2.dropna()
m['condition_loc'] = m['condition_loc'].astype(str)

xlim_adjusted = (-0.5, len(stats_subset['condition'].unique())-0.5+buffer*(len(gene_order)))
scatter_kwargs2 = dict(s=4, jitter=0.1, linewidth=0.5, edgecolor='white', native_scale=True)

In [ ]:
subfig = subfigures['H']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.55, top=0.35, right=0.15)
axes = subfig.subplots(1,4, gridspec_kw=dict(wspace=0.25))

plot_df = stats_subset

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('condition', sort=False):
    sns.stripplot(data=group, x='condition_loc', y='output_gmean', hue='condition', palette=condition_palette,
                    legend=False, ax=ax, marker=condition_markers[construct], **scatter_kwargs2)
ax.set(title='Mean', xlim=xlim_adjusted, xlabel='', ylabel='', yscale='log', xticks=xtick_locs)
marker_baseline = stats2.loc[(stats2['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('condition', sort=False):
    sns.stripplot(data=group, x='condition_loc', y='output_std', hue='condition', palette=condition_palette,
                    legend=False, ax=ax, marker=condition_markers[construct], **scatter_kwargs2)
ax.set(title='Standard deviation', xlim=xlim_adjusted, xlabel='', ylabel='', yscale='log', xticks=xtick_locs)

# slope
ax = axes[2]
for construct, group in plot_df.groupby('condition', sort=False):
    sns.stripplot(data=group, x='condition_loc', y='slope', hue='condition', palette=condition_palette,
                    legend=False, ax=ax, marker=condition_markers[construct], **scatter_kwargs2)
ax.set(title='Slope', xlim=xlim_adjusted, xlabel='', ylabel='', xticks=xtick_locs,)

# CV
ax = axes[3]
for construct, group in plot_df.groupby('condition', sort=False):
    sns.stripplot(data=group, x='condition_loc', y='output_variation', hue='condition', palette=condition_palette,
                    legend=False, ax=ax, marker=condition_markers[construct], **scatter_kwargs2)
ax.set(title='CV', xlim=xlim_adjusted, xlabel='', ylabel='', xticks=xtick_locs, ylim=(0,4))

for i,ax in enumerate(axes):
    yloc = -6
    ax.axvspan(4.5+buffer/2, 6.5+buffer*1.5, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
    ax.axvspan(9.5+buffer*2.5, 12.5+buffer*3.5, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
    ax.annotate(architecture_order[0], (2,yloc), xycoords=('data','axes points'), ha='center', va='top', ma='center', fontsize=smaller_size)
    ax.annotate(architecture_order[1], (5.5+buffer,yloc), xycoords=('data','axes points'), ha='center', va='top', ma='center', fontsize=smaller_size)
    ax.annotate('2V\nEF1a', (8+buffer*2,yloc), xycoords=('data','axes points'), ha='center', va='top', ma='center', fontsize=smaller_size)
    ax.annotate('2V\nU6', (11+buffer*3,yloc), xycoords=('data','axes points'), ha='center', va='top', ma='center', fontsize=smaller_size)
    ax.set_xticklabels(['']*len(ax.get_xticklabels()))
    sns.despine(ax=ax)

fig.savefig(rd.outfile(output_path))

In [ ]:
# Save to OneDrive
fig.savefig(rd.outfile(rd.datadir/'manuscripts'/'2024_miR-iFFL'/'figures'/'links'/'fig_designs-supp.pdf'))